In [75]:
# import dependencies
import py_midicsv as pm


In [76]:
# define functions

# rescale between two ranges of values
def scale(OldValue, OldMin, OldMax, NewMin, NewMax):
    NewValue = (((OldValue - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
    return NewValue

# convert time in ms to time in however the midi file wants it
def ms2midi(ms):
    new_ms = scale(ms, 0, 500, 0, 480)
    return new_ms

# convert seconds and frames to ms
def frame2ms(t):
    return int(t)*1000 + round(round((t - int(t))*100)*(1000/framerate))


In [77]:
# set parameters

# names for files to be imported and exported
og_midi = "example.mid"
new_midi = "remappedexample.mid"
convert_csv = "example.csv"

# set framerate
framerate = 28.67

# set bpm
bpm = 120


In [78]:
# convert midi data to csv format and retrive note times

# Load the MIDI file and parse it into csv format
csv_string = pm.midi_to_csv(og_midi)

# list to hold note times
times = []

# extract not times as integers from string
for line in csv_string:
    if 'Note_' in line or 'End_track' in line:
        time = int(line.split(',')[1][1:])
        times.append(time)

# note times relative to first note in midi track (not relative to begining of song file)
times = list(map(lambda t: t - times[0], times))



In [79]:
# get midi timings for timestamps

# list of step timings
timings = [
    0.01, 0.25, 1.20, 2.10, 3.00, 3.28, 4.21, 5.11, 6.08, 7.02, 8.04, 8.26, 9.19, 10.13,
    11.05, 12.03, 12.27, 13.16, 14.11, 15.05, 15.27, 16.21, 17.17, 18.10, 19.03, 20.00, 
    20.23, 21.26, 23.12, 24.14, 25.10, 26.02, 27.01, 27.25, 28.19, 29.12, 30.08, 31.02, 
    31.19, 32.11, 33.06, 34.01, 34.21, 35.15, 36.10, 37.04, 37.25, 38.20, 39.14, 40.10, 
    41.01, 41.27, 42.21, 43.14, 44.08, 45.01, 45.24, 46.22, 47.12, 48.04
]

# convert seconds and frames to ms
timings = list(map(frame2ms, timings))


In [80]:
# remap notes in midi file to synchronise with timestamps as though they were quarter notes

# which note in csv_string
which = 0

# rescaled times for each note
new_times = []

# get duration of quarter note in ms according to set bpm
bpm_ms = ms2midi(60000/bpm)

# loop through note times
for time in times:
    
    # calculate how many quarter notes from the beginning the note is
    beat = int(time/bpm_ms)
    
    # origninal minimum and maximum for scaling
    omin = beat*bpm_ms
    omax = (beat+1)*bpm_ms
    
    # new minimum and maximum for scaling
    try:
        nmin = timings[beat]
        nmax = timings[beat+1]
    
    # if note is on a beat that exceeds the number of timestamps
    except IndexError:
        
        # new min is second to last step new max is last step
        nmin = timings[len(timings)-2]
        nmax = timings[len(timings)-1]
    
    # rescaled time in string format
    new_time = ' ' + str(int(scale(time, omin, omax, nmin, nmax)))
    
    # add rescaled time
    new_times.append(new_time)


In [81]:
# note being referenced during csv parsing
which = 0

# iterate through lines in csv
for i, line in enumerate(csv_string):
    
    # replace old note timings with new note timings
    if 'Note_' in line or 'End_track' in line:
        new_line = line.split(',')
        new_line[1] = new_times[which]
        csv_string[i] = ','.join(map(str,new_line))
        which += 1


# display csv with new timings as a string
csv_string
        

['0, 0, Header, 0, 1, 480\n',
 '1, 0, Start_track\n',
 '1, 0, Channel_prefix, 0\n',
 '1, 0, Title_t, "Classic Electric Piano"\n',
 '1, 0, Instrument_name_t, "Spectral Pizzicato"\n',
 '1, 0, Time_signature, 4, 2, 24, 8\n',
 '1, 0, Key_signature, 0, "major"\n',
 '1, 0, SMPTE_offset, 33, 0, 0, 0, 0\n',
 '1, 0, Tempo, 500000\n',
 '1, 35, Note_on_c, 0, 55, 93\n',
 '1, 450, Note_off_c, 0, 55, 64\n',
 '1, 872, Note_on_c, 0, 58, 93\n',
 '1, 1281, Note_off_c, 0, 58, 64\n',
 '1, 1698, Note_on_c, 0, 51, 93\n',
 '1, 2020, Note_off_c, 0, 51, 64\n',
 '1, 2349, Note_on_c, 0, 54, 93\n',
 '1, 2671, Note_off_c, 0, 54, 64\n',
 '1, 3000, Note_on_c, 0, 57, 93\n',
 '1, 3484, Note_off_c, 0, 57, 64\n',
 '1, 3977, Note_on_c, 0, 55, 93\n',
 '1, 4351, Note_off_c, 0, 55, 64\n',
 '1, 4732, Note_on_c, 0, 52, 93\n',
 '1, 5055, Note_off_c, 0, 52, 64\n',
 '1, 5384, Note_on_c, 0, 57, 93\n',
 '1, 5827, Note_off_c, 0, 57, 64\n',
 '1, 6279, Note_on_c, 0, 55, 93\n',
 '1, 6671, Note_off_c, 0, 55, 64\n',
 '1, 7070, Note_on_c

In [82]:
# write new note times in string format to csv file
with open(convert_csv, "w") as f:
    f.writelines(csv_string)

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open(new_midi, "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)
